In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Tablas

In [ ]:
Namings = {
'load_date':'FechaProceso',
'opening_date':'FechaApertura',
'last_expiry_date':'FechaVencimiento',
'contract_cancel_date':'FechaCancelacion',
'account_opening_date':'FechaApertura',
'original_expiry_date':'FechaVencimiento',
'cutoff_date':'FechaProceso',
'start_date':'FechaAperturaMarco',
'expiration_date':'FechaVencimientoMarco',
'cancel_date':'FechaCancelacionMarco',
'contract_id':'ContratoDispo',
'source_contract_id':'ContratoMarcoFicticio',
'source_contract1_id':'ContratoMarcoAplicativo',
'main_operation_id':'ContratoMarcoAplicativo',
'customer_id':'CodigoCliente',
'gl_account_id':'CuentaContable',
'commercial_product_id':'IdProd',
'commercial_subproduct_id':'IdSubprod',
'summarized_local_product_id':'ProductoCostos',
'sender_application_id':'Aplicativo',
'gl_account_branch_id':'Oficina',
'currency_id':'Divisa',
'origin_currency_balance_amount':'SaldoP',
'mthly_lcur_average_bal_amount':'SaldoM',
'current_bal_local_curncy_amount':'SaldoPO',
'mthly_lcur_average_bal1_amount':'SaldoMO',
'initial_amount':'MontoInicial',
'contract_rate_type':'TipoTasa',
'CD_PROD_COMER':'IdProd',
'CD_SUBPROD_COMER':'IdSubprod',
'PRODCOME':'NombreProducto',
'CODIGO_ALTAMIRA':'CodigoCliente',
'GRUPO':'GrupoCib'
}

ColNum = {'SaldoP':np.float64,
          'SaldoM':np.float64,
          'SaldoMO':np.float64,
          'SaldoPO':np.float64}

## Mafi

In [ ]:
MafiActual = pd.read_csv('MafiActual.csv',dtype=str).rename(columns=Namings).astype(ColNum)
MafiAnterior = pd.read_csv('MafiAnterior.csv',dtype=str).rename(columns=Namings).astype(ColNum)

## Atributos

In [ ]:
AccActual = pd.read_csv('AccActual.csv',dtype=str).rename(columns=Namings)
AttActual = pd.read_csv('AttActual.csv',dtype=str).rename(columns=Namings)
CardActual = pd.read_csv('CardActual.csv',dtype=str).rename(columns=Namings)
AccAnterior = pd.read_csv('AccAnterior.csv',dtype=str).rename(columns=Namings)
AttAnterior = pd.read_csv('AttAnterior.csv',dtype=str).rename(columns=Namings)
CardAnterior = pd.read_csv('CardAnterior.csv',dtype=str).rename(columns=Namings)
ListaDfAtributos = [AccActual,AttActual,CardActual,AccAnterior,AttAnterior,CardAnterior]
Atributos = pd.concat(ListaDfAtributos)
Atributos['FechaProceso'] = Atributos['FechaProceso'].str[:7]

# Rorc

In [ ]:
RorcActual = pd.read_csv('RorcActual.csv',dtype=str).rename(columns=Namings)
RorcAnterior = pd.read_csv('RorcAnterior.csv',dtype=str).rename(columns=Namings)
Rorc = pd.concat([RorcActual,RorcAnterior])
Rorc['FechaProceso'] = Rorc['FechaProceso'].str[:7]

# OppAtt

In [ ]:
OpAttActual = pd.read_csv('OpAttActual.csv',dtype=str).rename(columns=Namings)
OpAttAnterior = pd.read_csv('OpAttAnterior.csv',dtype=str).rename(columns=Namings)
OpAtt = pd.concat([OpAttActual,OpAttAnterior])
OpAtt['FechaProceso'] = OpAtt['FechaProceso'].str[:7]

# Productos

In [ ]:
Productos = pd.read_csv('CatProdCom.csv',dtype=str,encoding='latin-1',
                        usecols=['CD_PROD_COMER','CD_SUBPROD_COMER','PRODCOME']).rename(columns=Namings)

# Catalogo Clientes

In [ ]:
Clientes = pd.read_csv('CatClientes.csv',dtype=str,encoding='latin-1').rename(columns=Namings)

----

# Proceso

In [ ]:
Mafi = MafiActual.merge(MafiAnterior, on = ['ContratoDispo', 'CodigoCliente', 'CuentaContable',
                                            'IdProd', 'IdSubprod',
                                            'Aplicativo', 'Oficina', 'Divisa'],
                                            how='outer',
                                            suffixes=('Actual', 'Anterior'))
Mafi['FechaProceso'] = Mafi['FechaProcesoActual'].fillna(Mafi['FechaProcesoAnterior'])
Mafi = Mafi.drop(['FechaProcesoActual','FechaProcesoAnterior'],axis=1)
cols = ['FechaProceso'] + [col for col in Mafi.columns if col != 'FechaProceso']
Mafi = Mafi[cols]
Mafi['FechaProceso'] = Mafi['FechaProceso'].str[:7]
Mafi[Mafi.select_dtypes(include='number').columns] = Mafi.select_dtypes(include='number').fillna(0)
Mafi['FechaMes'] = MafiActual['FechaProceso']
Mafi.shape

In [ ]:
Mafi['VarSaldoM'] = Mafi['SaldoMActual'] - Mafi['SaldoMAnterior']
Mafi['VarSaldoP'] = Mafi['SaldoPActual'] - Mafi['SaldoPAnterior']
Mafi['VarSaldoMO'] = Mafi['SaldoMOActual'] - Mafi['SaldoMOAnterior']
Mafi['VarSaldoPO'] = Mafi['SaldoPOActual'] - Mafi['SaldoPOAnterior']

In [ ]:
Mafi1 = Mafi.merge(Atributos,
                   on = ['ContratoDispo','FechaProceso'],
                   how = 'left').merge(Rorc,
                                       on = ['ContratoDispo','FechaProceso'],
                                       how = 'left').merge(OpAtt,
                                                           on=['ContratoMarcoAplicativo','FechaProceso'],
                                                           how='left')
Mafi1.shape
ColumnasFecha = ['FechaProceso','FechaApertura','FechaVencimiento','FechaCancelacion',
                 'FechaAperturaMarco', 'FechaVencimientoMarco','FechaMes']
Mafi1[ColumnasFecha] = Mafi1[ColumnasFecha].apply(lambda x: x.str[:7])
Mafi1[ColumnasFecha] = Mafi1[ColumnasFecha].apply(lambda x: pd.to_datetime(x, format='%Y-%m'))

In [ ]:
# Campo Entrada
Mafi1['Entrada'] = np.where(
    (Mafi1['FechaProceso'].dt.year == Mafi1['FechaApertura'].dt.year) &
    (Mafi1['FechaProceso'].dt.month == Mafi1['FechaApertura'].dt.month),
    'alta',
    None)

# Campo Salida

Mafi1['Salida'] = np.where(
    (Mafi1['FechaVencimiento'].dt.year == Mafi1['FechaCancelacion'].dt.year) & 
    (Mafi1['FechaVencimiento'].dt.month == Mafi1['FechaCancelacion'].dt.month)&
    (Mafi1['FechaVencimiento'].dt.year == Mafi1['FechaCancelacion'].dt.year) & 
    (Mafi1['FechaVencimiento'].dt.month == Mafi1['FechaCancelacion'].dt.month),
    'natural',
    np.where(
        (Mafi1['FechaApertura'].dt.year == Mafi1['FechaCancelacion'].dt.year) & 
        (Mafi1['FechaApertura'].dt.month == Mafi1['FechaCancelacion'].dt.month) &
        (Mafi1['FechaApertura'].dt.year == Mafi1['FechaMes'].dt.year) & 
        (Mafi1['FechaApertura'].dt.month == Mafi1['FechaMes'].dt.month),
        'alta-baja',
        np.where(
            ((Mafi1['FechaCancelacion'].dt.year == Mafi1['FechaProceso'].dt.year) & 
            (Mafi1['FechaCancelacion'].dt.month == Mafi1['FechaProceso'].dt.month))&
            (Mafi1['FechaCancelacion'].dt.year < Mafi1['FechaVencimiento'].dt.year) |
            ((Mafi1['FechaCancelacion'].dt.year == Mafi1['FechaVencimiento'].dt.year) &
             (Mafi1['FechaCancelacion'].dt.month < Mafi1['FechaVencimiento'].dt.month)),
            'prepago',
            None
        )
    )
)


# Campo linea
Mafi1['Linea'] = np.select(
        [
            Mafi1['ContratoMarcoAplicativo'].isnull() & Mafi1['ContratoMarcoFicticio'].notnull(),
            Mafi1['ContratoMarcoFicticio'].isnull() & Mafi1['ContratoMarcoAplicativo'].notnull(),
            Mafi1['ContratoMarcoFicticio'].notnull() & Mafi1['ContratoMarcoAplicativo'].notnull(),
            Mafi1['ContratoMarcoFicticio'].isnull() & Mafi1['ContratoMarcoAplicativo'].isnull()
        ],
        [
            'comprometida',
            'no comprometida',
            'comprometida',
            'no linea'
        ],
        default=None
    )

# Campo New Money
Mafi1['NewMoney'] = np.where(
    (Mafi1['FechaAperturaMarco'].dt.year == Mafi1['FechaApertura'].dt.year) & 
    (Mafi1['FechaAperturaMarco'].dt.month == Mafi1['FechaApertura'].dt.month)&
    (Mafi1['FechaAperturaMarco'].dt.year == Mafi1['FechaProceso'].dt.year) & 
    (Mafi1['FechaAperturaMarco'].dt.month == Mafi1['FechaProceso'].dt.month),
    '1',
    np.where(
        (Mafi1['Linea'] == 'no linea') &
        ((Mafi1['FechaApertura'].dt.year == Mafi1['FechaMes'].dt.year) & 
        (Mafi1['FechaApertura'].dt.month == Mafi1['FechaMes'].dt.month)),
        '2',
        None
    )
)


In [ ]:
Mafi1 = Mafi1.merge(Productos,
                   on = ['IdProd','IdSubprod'],
                   how = 'left').merge(Clientes,
                                       on = 'CodigoCliente',
                                       how = 'left')

In [ ]:
# Validacion Salida: En el Saldo Actual deben ser todos 0 y puede haber cifras en el saldo del mes pasado
(pd.pivot_table(Mafi1,
               index = ['Salida','ContratoDispo'],
               values=['SaldoPActual','SaldoPAnterior'],
               dropna=False,
               fill_value=0,
               margins=True,
               aggfunc='sum')/1000000).head()

In [ ]:
pd.pivot_table(Mafi1,
               index = ['NewMoney'],
               values=['SaldoPActual','SaldoPAnterior'],
               dropna=False,
               fill_value=0,
               margins=True,
               aggfunc='sum')/1000000

In [ ]:
Mafi1.to_excel('Enero24_v2.xlsx',index=False)